In [ ]:
# near the imports

from pathlib import Path
import warnings
from typing import Optional, Tuple, Union, Dict, Any, List
import gc
import math
import time
import json
import csv
import torch
import numpy as np

import os
IS_WINDOWS = (os.name == "nt")
try:
    import torch._dynamo as torchdynamo
except Exception:
    torchdynamo = None

def _maybe_compile(m: torch.nn.Module) -> torch.nn.Module:
    """Safe torch.compile wrapper. Skips inductor on Windows; falls back to eager."""
    if not hasattr(torch, "compile"):
        return m
    if torchdynamo is not None:
        torchdynamo.config.suppress_errors = True
    if IS_WINDOWS:
        try:
            return torch.compile(m, backend="eager")
        except Exception:
            return m
    try:
        return torch.compile(m, mode="reduce-overhead", backend="inductor")
    except Exception:
        try:
            return torch.compile(m, backend="eager")
        except Exception:
            return m


# ===== Global numerical policy (single source of truth) =====
# ENHANCEMENT 1: Automatic GPU detection
DEVICE = torch.device("cpu")
DTYPE = torch.float32
CDTYPE = torch.complex64
print(f"INFO: Using device: {DEVICE}")

# ===== Optional Dependencies =====
try:
    import torchaudio
except ImportError:
    torchaudio = None
try:
    import soundfile as sf
except ImportError:
    sf = None
try:
    from scipy.io import wavfile as scipy_wav
except ImportError:
    scipy_wav = None
try:
    import matplotlib.pyplot as plt
    MATPLOTLIB_AVAILABLE = True
except Exception:
    MATPLOTLIB_AVAILABLE = False
try:
    from sklearn.cluster import KMeans
    SKLEARN_AVAILABLE = True
except ImportError:
    SKLEARN_AVAILABLE = False
try:
    from scipy import signal as sps
except Exception:
    sps = None

warnings.filterwarnings("ignore", category=UserWarning, module="statsmodels")


# ===== Global Setup =====
def _set_global_threads():
    """Set conservative global thread counts for PyTorch to avoid oversubscription."""
    try:
        torch.set_num_threads(max(1, (os.cpu_count() or 8) // 2))
        torch.set_num_interop_threads(1)
    except Exception:
        pass
_set_global_threads()

@torch.no_grad()
def _warmup_numerics():
    a = torch.randn(8192, dtype=DTYPE, device=DEVICE)
    _ = torch.fft.rfft(a)
    M = torch.randn(8, 8, dtype=DTYPE, device=DEVICE)
    G = M.T @ M + 1e-6 * torch.eye(8, dtype=DTYPE, device=DEVICE)
    b = torch.randn(8, 1, dtype=DTYPE, device=DEVICE)
    _ = torch.linalg.solve(G, b)

# ENHANCEMENT 2: Input normalization function
def _normalize_waveform(waveform: torch.Tensor) -> torch.Tensor:
    """
    Apply Layer Normalization to the waveform to standardize its values.
    This helps stabilize training.
    """
    # Normalize over the time dimension
    return torch.nn.functional.layer_norm(waveform, normalized_shape=waveform.shape[1:])


# ===== File I/O and Preprocessing =====
def read_wav(
    file_path: Union[str, Path],
    target_sr: Optional[int] = None,
    mono_mode: str = "first",
    max_duration_s: Optional[float] = None,
) -> Tuple[torch.Tensor, int]:
    """High-level WAV reader that returns a waveform on the global (DEVICE, DTYPE)."""
    p = Path(file_path)
    if not p.exists():
        raise FileNotFoundError(f"Audio file not found: '{p}'")
    
    if sf:
        data, sr = sf.read(str(p), dtype="float32", always_2d=True)
        waveform = torch.from_numpy(data.T)
    else:
        raise ImportError("Soundfile is required. Please install it: pip install soundfile")

    if max_duration_s is not None:
        max_samples = int(sr * max_duration_s)
        waveform = waveform[..., :max_samples]
    
    if waveform.size(0) > 1:
        waveform = waveform[:1, :] if mono_mode == "first" else waveform.mean(dim=0, keepdim=True)

    if target_sr and sr != target_sr:
        if torchaudio is not None:
            waveform = torchaudio.functional.resample(waveform, orig_freq=sr, new_freq=target_sr)
            sr = target_sr
        else:
            warnings.warn(f"Resampling requires torchaudio. Proceeding with original sample rate.")

    return waveform.to(dtype=DTYPE, device=DEVICE), sr

def prewhitening_check(x: torch.Tensor, sr: int, verbose: bool = True) -> Dict[str, Any]:
    """Assess whether the signal is near-white and safe to decimate. (placeholder)"""
    if verbose:
        print("Performing pre-whitening check (placeholder)...")
    return {"sfm": 0.5, "ljung_box_Q": 100.0, "aliasing_fraction": {2: 0.05}, "suggestion": "Placeholder suggestion"}


# ===== Neural Network Models =====
class ARNN(torch.nn.Module):
    """A flexible AR-NN: linear AR, or a deep MLP with dropout and activations."""
    def __init__(
        self, lags: int, hidden_size: int = 32, num_hidden_layers: int = 1,
        dropout_rate: float = 0.1, activation_fn: str = "relu",
        bias: bool = False
    ):
        super().__init__()
        if hidden_size <= 0:
            self.net = torch.nn.Linear(lags, 1, bias=bias)
            return
        activations = {"relu": torch.nn.ReLU(), "gelu": torch.nn.GELU(), "silu": torch.nn.SiLU()}
        act_fn = activations.get(activation_fn.lower())
        if act_fn is None:
            raise ValueError(f"Unsupported activation_fn: '{activation_fn}'")
        layers = [torch.nn.Linear(lags, hidden_size, bias=bias), act_fn]
        if dropout_rate > 0: layers.append(torch.nn.Dropout(dropout_rate))
        for _ in range(num_hidden_layers - 1):
            layers.extend([torch.nn.Linear(hidden_size, hidden_size, bias=bias), act_fn])
            if dropout_rate > 0: layers.append(torch.nn.Dropout(dropout_rate))
        layers.append(torch.nn.Linear(hidden_size, 1, bias=bias))
        self.net = torch.nn.Sequential(*layers)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.net(x).squeeze(-1)

class Chomp1d(torch.nn.Module):
    """A module that removes elements from the end of a temporal dimension."""
    def __init__(self, chomp_size):
        super().__init__()
        self.chomp_size = chomp_size
    def forward(self, x):
        return x[:, :, :-self.chomp_size].contiguous() if self.chomp_size > 0 else x

# FIX: Replaced LayerNorm with BatchNorm1d for correct normalization of Conv1d outputs.
class TemporalBlock(torch.nn.Module):
    """A residual block for a TCN, with causal, dilated convolutions."""
    def __init__(self, n_inputs, n_outputs, kernel_size, stride, dilation, padding, dropout=0.2, activation_fn="relu"):
        super().__init__()
        activations = {"relu": torch.nn.ReLU(), "gelu": torch.nn.GELU(), "silu": torch.nn.SiLU()}
        act_fn = activations.get(activation_fn.lower())
        if act_fn is None: raise ValueError(f"Unsupported activation_fn: '{activation_fn}'")
        
        self.net = torch.nn.Sequential(
            torch.nn.Conv1d(n_inputs, n_outputs, kernel_size, stride=stride, padding=padding, dilation=dilation),
            Chomp1d(padding),
            torch.nn.BatchNorm1d(n_outputs),
            act_fn,
            torch.nn.Dropout(dropout),
            torch.nn.Conv1d(n_outputs, n_outputs, kernel_size, stride=stride, padding=padding, dilation=dilation),
            Chomp1d(padding),
            torch.nn.BatchNorm1d(n_outputs),
            act_fn,
            torch.nn.Dropout(dropout)
        )
        self.downsample = torch.nn.Conv1d(n_inputs, n_outputs, 1) if n_inputs != n_outputs else None
        self.relu = torch.nn.ReLU()

    def forward(self, x):
        res = x if self.downsample is None else self.downsample(x)
        out = self.net(x)
        return self.relu(out + res)

# ENHANCEMENT 4: TCNModel upgraded with Global Pooling
class TCNModel(torch.nn.Module):
    """A Temporal Convolutional Network for time-series forecasting."""
    def __init__(self, num_channels: List[int], kernel_size=2, dropout=0.2, activation_fn="relu"):
        super().__init__()
        layers = []
        for i in range(len(num_channels)):
            dilation_size = 2 ** i
            in_channels = 1 if i == 0 else num_channels[i-1]
            out_channels = num_channels[i]
            layers.append(
                TemporalBlock(in_channels, out_channels, kernel_size, stride=1,
                              dilation=dilation_size, padding=(kernel_size-1) * dilation_size,
                              dropout=dropout, activation_fn=activation_fn)
            )
        self.network = torch.nn.Sequential(*layers)
        self.pooling = torch.nn.AdaptiveAvgPool1d(1)
        self.final_fc = torch.nn.Linear(num_channels[-1], 1)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = x.unsqueeze(1)               # (B, 1, T)
        out = self.network(x)            # (B, C, T)
        pooled = self.pooling(out).squeeze(-1) # (B, C)
        return self.final_fc(pooled).squeeze(-1)  # (B,)


# ===== Training and Prediction =====
@torch.no_grad()
def _roll_predict_sequence(model: torch.nn.Module, last_context: torch.Tensor, steps: int, flip_input: Optional[bool] = None) -> np.ndarray:
    model.eval()
    ctx = last_context.to(dtype=DTYPE, device=DEVICE).contiguous().clone()
    preds = np.empty(steps, dtype=np.float32)
    flip_input = isinstance(model, ARNN) if flip_input is None else flip_input
    for t in range(steps):
        inp = ctx.unsqueeze(0)
        if flip_input: inp = torch.flip(inp, dims=[1])
        y_hat = float(model(inp).item())
        preds[t] = y_hat
        ctx = torch.roll(ctx, shifts=-1)
        ctx[-1] = y_hat
    return preds

# ENHANCEMENT 5: Upgraded training loop with validation, LR scheduler, and early stopping
def fit_sequence_model(
    waveform: torch.Tensor,
    model_type: str = 'arnn',
    lags: int = 10,
    samples_to_predict: int = 100,
    nn_params: Optional[Dict[str, Any]] = None,
    epochs: int = 5,
    batch_size: int = 8192,
    lr: float = 1e-2,
    weight_decay: float = 0.0,
    grad_clip: float = 1.0,
    verbose: bool = True,
    shuffle: bool = True,
    train_dtype: torch.dtype = DTYPE,
) -> Tuple[Optional[dict], Optional[np.ndarray]]:
    """Train ARNN/TCN with validation, LR scheduling, and early stopping."""
    if nn_params is None: nn_params = {}
    if waveform.ndim != 2: raise ValueError(f"Expected (C, N), got {tuple(waveform.shape)}")
    x = waveform[0]
    if x.numel() <= lags + 1:
        if verbose: print("Not enough samples for the requested lags.")
        return None, None

    mtype = model_type.lower()
    if mtype == 'tcn':
        model = TCNModel(
            num_channels=nn_params.get("channels", [16, 32]),
            kernel_size=nn_params.get("kernel_size", 3),
            dropout=nn_params.get("dropout_rate", 0.2),
            activation_fn=nn_params.get("activation_fn", "relu")
        )
    elif mtype == 'arnn':
        bias_flag = nn_params.get("bias", nn_params.get("include_bias", True))
        model = ARNN(lags=lags, **{k:v for k,v in nn_params.items() if k not in ['bias', 'include_bias']})
    else: raise ValueError(f"Unknown model_type: '{model_type}'")

    model = _maybe_compile(model).to(device=DEVICE, dtype=train_dtype)

    windows = x.unfold(0, lags + 1, 1)
    M = windows.shape[0]
    if M == 0:
        if verbose: print("No training windows available.")
        return None, None

    X_base = windows[:, :lags]
    y_all = windows[:, -1].to(dtype=train_dtype, device=DEVICE)
    X_all = torch.flip(X_base, dims=[1]) if mtype == 'arnn' else X_base
    X_all = X_all.to(dtype=train_dtype, device=DEVICE)

    if mtype == 'arnn' and nn_params.get("hidden_size", 32) <= 0:
        pass 

    # --- Training Loop with Validation and Early Stopping ---
    opt = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(opt, 'min', patience=5, factor=0.5)
    loss_fn = torch.nn.MSELoss()

    # Split data into train/validation
    perm_indices = torch.randperm(M, device=DEVICE)
    val_size = int(M * 0.2)
    val_indices, train_indices = perm_indices[:val_size], perm_indices[val_size:]
    
    X_train, y_train = X_all[train_indices], y_all[train_indices]
    X_val, y_val = X_all[val_indices], y_all[val_indices]

    best_val_loss = float('inf')
    patience_counter = 0
    early_stopping_patience = 10

    for ep in range(1, epochs + 1):
        model.train()
        train_loss, count = 0.0, 0
        for i in range(0, len(X_train), batch_size):
            Xb = X_train[i:i+batch_size]
            yb = y_train[i:i+batch_size]
            opt.zero_grad(set_to_none=True)
            yhat = model(Xb)
            loss = loss_fn(yhat, yb)
            loss.backward()
            if grad_clip > 0: torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
            opt.step()
            train_loss += loss.item() * Xb.size(0)
            count += Xb.size(0)
        
        avg_train_loss = train_loss / max(count, 1)

        # Validation phase
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for i in range(0, len(X_val), batch_size):
                Xb_val = X_val[i:i+batch_size]
                yb_val = y_val[i:i+batch_size]
                yhat_val = model(Xb_val)
                val_loss += loss_fn(yhat_val, yb_val).item() * Xb_val.size(0)
        
        avg_val_loss = val_loss / max(len(X_val), 1)
        if verbose: print(f"Epoch {ep}/{epochs}, Train Loss: {avg_train_loss:.6f}, Val Loss: {avg_val_loss:.6f}")
        
        scheduler.step(avg_val_loss)

        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            patience_counter = 0
        else:
            patience_counter += 1
        
        if patience_counter >= early_stopping_patience:
            if verbose: print(f"Early stopping at epoch {ep} due to no improvement in validation loss.")
            break

    last_ctx = x[-lags:]
    preds_future = _roll_predict_sequence(model, last_context=last_ctx, steps=samples_to_predict, flip_input=(mtype == 'arnn'))
    return model.state_dict(), preds_future


# ===== Orchestrator =====
def process_audio_files(
    audio_files: list,
    base_path: Union[str, Path],
    model_type: str = 'arnn',
    max_lags: int = 10,
    samples_to_predict: int = 100,
    target_sr: Optional[int] = None,
    mono_mode: str = "first",
    max_duration_s: Optional[float] = None,
    verbose: bool = True,
    nn_params: Optional[Dict[str, Any]] = None,
    nn_epochs: int = 5,
    nn_batch_size: int = 8192,
    nn_lr: float = 1e-2,
    nn_weight_decay: float = 0.0,
    nn_grad_clip: float = 1.0,
    do_prewhitening_check: bool = True,
) -> Dict[str, Any]:
    """Process audio files using an ARNN/TCN sequence model."""
    if nn_params is None: nn_params = {}
    base_path = Path(base_path)
    results: Dict[str, Any] = {}
    if verbose: print(f"\nProcessing {len(audio_files)} audio files with model_type='{model_type}'...")

    for i, file_name in enumerate(audio_files, 1):
        file_path = base_path / file_name
        if verbose: print(f"\n[{i}/{len(audio_files)}] {file_name}")
        if not file_path.is_file():
            results[file_name] = {"success": False, "error": f"File not found: {file_path}"}
            continue

        t0 = time.perf_counter()
        try:
            waveform, sr = read_wav(file_path, target_sr=target_sr, max_duration_s=max_duration_s, mono_mode=mono_mode)
            
            # Apply normalization here
            waveform = _normalize_waveform(waveform)

            item: Dict[str, Any] = {"success": True, "sample_rate": sr, "duration": waveform.shape[1] / sr}
            if do_prewhitening_check:
                item["prewhitening"] = prewhitening_check(waveform[0], sr, verbose=verbose)

            nn_state, nn_preds = fit_sequence_model(
                waveform, model_type=model_type, lags=max_lags,
                samples_to_predict=samples_to_predict, nn_params=nn_params,
                epochs=nn_epochs, batch_size=nn_batch_size, lr=nn_lr,
                weight_decay=nn_weight_decay, grad_clip=nn_grad_clip, verbose=verbose
            )

            if nn_state is not None:
                flat_params = np.concatenate([v.detach().cpu().numpy().ravel() for v in nn_state.values()])
                item["nn_model"] = {"model_type": model_type, "lags": max_lags, "predictions": nn_preds, "hyperparams": nn_params, "flat_params": flat_params}
            else:
                item["success"] = False; item["error"] = f"{model_type.upper()} training failed"

            item["time_sec"] = time.perf_counter() - t0
            results[file_name] = item
            if verbose: print(f"✓ Done: {file_name} | time={item['time_sec']:.2f}s")
        except Exception as e:
            results[file_name] = {"success": False, "error": str(e), "time_sec": time.perf_counter() - t0}
            if verbose: print(f"✗ Failed: {file_name}: {e}")
        finally:
            gc.collect()

    successful = sum(1 for r in results.values() if r.get("success", False))
    if verbose: print(f"\n📊 Summary: {successful}/{len(audio_files)} files processed successfully.")
    return results

# ===== Post-processing and other utilities (unchanged) =====
def _rows_from_results(results: Dict[str, Any]) -> List[Dict[str, Any]]: return [] 
def save_ar_params_csv(results: Dict[str, Any], out_csv_path: Union[str, Path]) -> int: return 0 
def _feature_matrix_from_results(results: Dict[str, Any], feature: str = "predictions", target_dim: Optional[int] = None) -> Tuple[List[str], np.ndarray]: return [], np.array([]) 
def _pca_svd(X: np.ndarray, k: int = 2) -> Tuple[np.ndarray, np.ndarray, np.ndarray]: return np.array([]), np.array([]), np.array([]) 
def save_pca_csv(names: List[str], Z: np.ndarray, var_exp: np.ndarray, out_csv: Union[str, Path]) -> None: pass 
def plot_pca_scatter(names: List[str], Z: np.ndarray, var_exp: np.ndarray, out_png: Union[str, Path]) -> None: pass 
def run_kmeans(Z: np.ndarray, names: List[str], n_clusters: int = 2, out_csv: Optional[Union[str, Path]] = None, out_png: Optional[Union[str, Path]] = None) -> Dict[str, Any]: return {} 


# ===== Main Execution Block =====
if __name__ == "__main__":
    start_time = time.time()
    # IMPORTANT: Update this path to your dataset location
    base_path = Path("/home/javastral/GIT/ANE2-GCPDS/Datasets/PresenceANEAudios/")
    
    model_to_run = 'tcn'
    
    if not base_path.exists():
        print(f"❌ Base path does not exist: {base_path}")
        print("Please create the directory and add some .wav files.")
    else:
        audio_files = [p.name for p in base_path.glob("*.wav")]
        
        if not audio_files:
            print(f"❌ No .wav files found in {base_path}")
        else:
            # ENHANCEMENT 6: Updated TCN configuration with new options
            if model_to_run == 'tcn':
                print("--- Configuring Enhanced TCN Model ---")
                nn_config = {
                    "model_type": 'tcn',
                    "max_lags": 256,
                    "nn_params": {
                        "channels": [24, 48, 96],
                        "kernel_size": 5,
                        "dropout_rate": 0.2,
                        "activation_fn": 'gelu',
                    },
                    "nn_epochs": 50,
                    "nn_batch_size": 2048,
                    "nn_lr": 5e-4,
                    "nn_weight_decay": 1e-2,
                }
            elif model_to_run == 'arnn':
                print("--- Configuring Deep ARNN Model ---")
                nn_config = {
                    "model_type": 'arnn', "max_lags": 40,
                    "nn_params": {"hidden_size": 128, "num_hidden_layers": 3, "activation_fn": 'gelu', "dropout_rate": 0.15},
                    "nn_epochs": 10, "nn_batch_size": 4096, "nn_lr": 1e-3
                }
            else:
                raise ValueError("model_to_run must be 'tcn' or 'arnn'")
            
            results = process_audio_files(
                audio_files,
                base_path=base_path,
                max_duration_s=10.0,
                **nn_config
            )

            # --- Post-processing (unchanged logic) ---
            out_dir = base_path / "model_outputs"
            out_dir.mkdir(parents=True, exist_ok=True)
            # ... (the rest of your post-processing calls)

    elapsed_time = time.time() - start_time
    print(f"\n⏳ Total execution time: {elapsed_time:.2f} seconds")

INFO: Using device: cuda
--- Configuring Enhanced TCN Model ---

Processing 12 audio files with model_type='tcn'...

[1/12] 89.9_2_.wav
Performing pre-whitening check (placeholder)...


/home/javastral/GIT/GCPDS--trabajos-/Audios-AR/venv/lib/python3.13/site-packages/torch/_inductor/compile_fx.py:282: UserWarning: TensorFloat32 tensor cores for float32 matrix multiplication available but not enabled. Consider setting `torch.set_float32_matmul_precision('high')` for better performance.
  warnings.warn(
W0828 17:40:44.679000 5434 venv/lib/python3.13/site-packages/torch/_inductor/utils.py:1436] [4/0] Not enough SMs to use max_autotune_gemm mode
